In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import csv 
from bs4 import BeautifulSoup
from multiprocessing import Pool
import os

url_list = "https://www.asus.com/us/Motherboards/AllProducts/"
driver = webdriver.PhantomJS('D:\\Github\\MovieCrawler\\phantomjs.exe')
# get()方法會一直等到頁面被完全加載，然後才會繼續程序
driver.get(url_list)

# 把selenium的webdriver调用page_source函数在传入BeautifulSoup中，就可以用BeautifulSoup解析网页了
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")


# 打印頁面標題：XXXX年最新电影_第1页-BT电影天堂
print(driver.title)

### 抓所有主板規格的URL

In [ ]:
f = open('MB_SPEC_LINK.txt', 'w', encoding = 'UTF-8')    # 也可使用指定路徑等方式，如： C:\A.txt
div_list = soup.find_all('div', class_='product_level_1')
for div in div_list:
    li_list = div.find('ul', class_='unstyled all-model-list').contents
    for li in li_list:
        a = li.find('a')
        ##只要herf用中括號就可, 用herf = a.get_attribute('href')會跳error
        print(a['href'])
        f.write('https://www.asus.com' + a['href'] + 'specifications/' + '\n')
f.close() 


### 改用Dictionary來存網址, 避免重複

In [ ]:
Mb_URL_dic = {}
for div in div_list:
    li_list = div.find('ul', class_='unstyled all-model-list').contents
    for li in li_list:
        a = li.find('a')
        ##只要herf用中括號就可, 用herf = a.get_attribute('href')會跳error
        Mb_URL_dic[a.text] = ('https://www.asus.com' + a['href'] + 'specifications/')
Mb_URL_dic


### 抓SPEC

In [ ]:
# SPEC_URL_list=[]
# f = open("D:\\Github\\MovieCrawler\\MB_SPEC_LINK.txt","r", encoding = 'UTF-8')
# for line in f:
#     SPEC_URL_list.append(line[:-1])
# f.close()

   

# url_list = "https://www.asus.com/us/Motherboards/H110S1-CSM/specifications/"
# driver.get(url_list)
# html = driver.page_source
# soup = BeautifulSoup(html, "html.parser")
# soup_title_list = soup.find_all('span', class_='spec-item')
# soup_data_list = soup.find_all('div', class_='spec-data')
# MB_name = soup.find('h1', class_='page-title').text

# title_list =['Board name']
# data_list = [MB_name]

# for title in soup_title_list:
#     title_list.append(title.text)
# for data in soup_data_list:
#     data_list.append(data.text)

## create DataFrame with title list
df = pd.DataFrame()
# df = df.append(pd.Series(data_list,index=title_list),ignore_index=True)

# print(title_list)

for link in Mb_URL_dic.values():
    driver.get(link)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    soup_title_list = soup.find_all('span', class_='spec-item')
    soup_data_list = soup.find_all('div', class_='spec-data')
    MB_name = soup.find('h1', class_='page-title').text

    title_list =['Board name']
    data_list = [MB_name]

    for title in soup_title_list:
        title_list.append(title.text)
    for data in soup_data_list:
        data_list.append(data.text)


    if soup_title_list :
        soup_data_list = soup.find_all('div', class_='spec-data')
        MB_name = soup.find('h1', class_='page-title').text
        data_list= [MB_name]
        for data in soup_data_list:
            data_list.append(data.text)
        print(data_list)
        try :
            df = df.append(pd.Series(data_list,index=title_list),ignore_index=True)
        except ValueError:
            print("Error MB page : " + MB_name)
    else :
        continue
df

In [ ]:
##Dataframe to csv file

df.to_csv ("mb_spec.csv" , encoding = "utf-8")

In [ ]:
from pymongo import MongoClient
from datetime import datetime

uri = "mongodb://10.78.20.176:27017/"

client = MongoClient(uri)
db = client.test
db.authenticate("denny","1234")
print(db.name)
col = db.testcollection
print(col)
# col.insert_one({"name":'yanying','province':'江苏','age':25})

col.insert_many([
    {"name":'张三','province':'浙江','age':24},
    {"name":'张三1','province':'浙江1','age':25},
    {"name":'张三2','province':'浙江2','age':26},
    {"name":'张三3','province':'浙江3','age':28},
])

def show(collection):
    # 查詢
    for item in collection.find():
        print(item)

show(col)

In [ ]:
# mb_spec.csv
import json
def mongoimport(csv_path, db_name, coll_name, db_url='10.78.20.176', db_port=27017):
    """ Imports a csv file at path csv_name to a mongo colection
    returns: count of the documants in the new collection
    """
    client = MongoClient(db_url, db_port)
    db = client[db_name]
    db.authenticate("denny","1234")
    coll = db[coll_name]
    data = pd.read_csv(csv_path)
    payload = json.loads(data.to_json(orient='records'))
    coll.remove()
    coll.insert(payload)
    return coll.count()


mongoimport("mb_spec.csv", "test", "testcollection")

In [ ]:
def mongoexportcsv(csv_path, db_name, coll_name, db_url='10.78.20.176', db_port=27017):
    """ 
    """
    #connect mongodb
    client = MongoClient(db_url, db_port)
    db = client[db_name]
    db.authenticate("denny","1234")
    coll = db[coll_name]
    with open('output_j.json', 'w') as f:
        #clean file
        f.truncate()
        data = list()
        for item in coll.find():
            del item['_id']
            data.append(item)
        f.write(json.dumps(data))

    # json to csv
    # json文件
    json_file = open('output_j.json', 'r')
    # csv文件
    csv_file = open(csv_path, 'w',encoding="utf-8")
    # csv读写器
    csv_writer = csv.writer(csv_file)
    # 表头和正文
    data = json.load(json_file)
    # 表头
    sheet_title = data[0].keys()
    content_list = [dic.values() for dic in data]
    # 5.通过读写器 写入csv文件
    csv_writer.writerow(sheet_title)
    csv_writer.writerows(content_list)
    # 6.关闭文件
    json_file.close()
    csv_file.close()
    client.close()
    return coll.count()


mongoexportcsv("output.csv", "test", "testcollection")

